In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.cluster import KMeans
from collections import defaultdict
from scipy.spatial.distance import cdist
import statsmodels.api as sm
import jenkspy
import datetime as DT
from datetime import datetime

In [2]:
# 2 STORE, 2 Models in total, within each model, it has predict days' model
def data_split():
    data = pd.read_csv('./new_data.csv')
    # smooth res
    data['y'] = 0
    for s in data.Store.unique():
        s_data = data.loc[data.Store==s]
        for d in s_data.Dept.unique():
            s_d_data = s_data.loc[s_data.Dept==d]
            lowess = sm.nonparametric.lowess
            smooth = lowess(s_d_data.Weekly_Sales, range(len(s_d_data.Weekly_Sales)), 0.05, 10, return_sorted=False)
            data.loc[(data.Store == s) & (data.Dept == d), 'y'] = smooth
    # turn Dept to onehot
    data1 = data.loc[data.Store==20]
    data2 = data.loc[data.Store==33]
    data1.to_csv('./model1_data.csv', index=False,encoding='utf-8')
    data2.to_csv('./model2_data.csv', index=False,encoding='utf-8')

In [2]:
data1 = pd.read_csv('./model1_data.csv')
data2 = pd.read_csv('./model2_data.csv')

Question 3: compare XGBoost with LSTM

In [9]:
# generate sale data 7, 14, 21 days before as features
def generate_sale_features(df, forward_peroid): # either -7, -14 days if day is unit or -1, -2 if week is unit..
    columns = df.columns
    feature_set = []
    new_column_names = []
    for p in forward_peroid:
        for feature in columns:
            if p <= len(df):
                feature_set += [df.iloc[-p][feature]]
            else:
                feature_set += [-99999] # mark as nan
            new_column_names += [str(-p) + '_' + str(feature)]
    return feature_set, new_column_names

In [11]:
# when training and testing using mse or others?? , test which slide gap and history time make more sense?
def generate_backtest_data(df, test_date, time_gap, slide_peroid, slide_step, forward_peroid):
    '''
    y is array of **training data**, include first prediction day [t_n, t_{n-1}... t1] (t1, t2, .. t7): prediction day for eg., 
    predict future days means how many future days to predict using machine learning models 
    '''
    # features for backtest data (train, validation)
    X_train = [[] for _ in range(len(test_date))] # len(test_date)
    X_validation = [[] for _ in range(len(test_date))]
    Y_train = [[] for _ in range(len(test_date))]
    Y_validation = [[] for _ in range(len(test_date))]

    # last train date : array
    cc = 0
    last_test_day_date = [(datetime.strptime(t,'%Y-%m-%d') - DT.timedelta(days=time_gap)).strftime('%Y-%m-%d') for t in test_date]
    print(last_test_day_date)
    for i in range(len(test_date)): # Generate train data for each day model
        # for each dept, 
        for d in df.Dept.unique():
            df_d = df.loc[(df.Dept == d) & (df.Date <= last_test_day_date[i])]

            t = -1 # start from last test day, then last train day is last test day - time_gap    
            # use id from last_test_ady_index
            while -(t - slide_peroid) <= len(df_d):
                test = df_d.iloc[t-slide_peroid-1 : t+1]
                train = df_d.iloc[t-slide_peroid: t] # train data not included test data
                other_feature_cols = ['Weekly_Sales', 'IsHoliday', 'Temperature',
           'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4',
           'MarkDown5', 'CPI', 'Unemployment', 'Size', 'promotion',
           'Month', 'Year', 'day', 'dayofweek', 'y'] # TO GET forward days

                # X_train
                Xtrain_y_features = [np.min(train.y), np.max(train.y), np.mean(train.y), np.std(train.y), stats.skew(train.y), stats.kurtosis(train.y)]

                y_names = ['min_y', 'max_y', 'mean_y', 'std_y', 'skew_y', 'kurtosis_y']
                Xtrain_other_features, other_names = generate_sale_features(train[other_feature_cols],forward_peroid)
                # Xtest
                Xtest_y_features = [np.min(test.y), np.max(test.y), np.mean(test.y), np.std(test.y), stats.skew(test.y), stats.kurtosis(test.y)]

                Xtest_other_features, _ = generate_sale_features(test[other_feature_cols], forward_peroid)
                # Ytrain
                Y_train[i] += [df_d.iloc[t-1].y]
                # Ytest
                Y_validation[i] += [df_d.iloc[t].y]
                X_train[i] += [Xtrain_y_features + Xtrain_other_features]
                X_validation[i] += [Xtest_y_features + Xtest_other_features]

                feature_names = y_names + other_names  
                t -= slide_step 

    return X_train, X_validation, Y_train, Y_validation, feature_names

In [23]:
# generate train and validation data for data1 and data2, because they will be trained seperately
# predict ['2012-10-05', '2012-10-12', '2012-10-19', '2012-10-26'] sales, 
# make sure that all id in test data have historical data in train data
X_train1, X_validation1, Y_train1, Y_validation1, feature_names = generate_backtest_data(df = data1.loc[data1.Date <= data1.Date.unique()[-5]], # training data 9864 records 
                                                                                     test_date = data1.Date.unique()[-4:], 
                                                                                     time_gap = 4 * 7, # week as unit, 4 weeks: 28 days 
                                                                                     slide_peroid = 8, # each has 8 week range 
                                                                                     slide_step = 2, 
                                                                                     forward_peroid = [1,2,3]) # slide back 1,2,3 weeks

['2012-09-07', '2012-09-14', '2012-09-21', '2012-09-28']


In [149]:
# generate predict days feature and label

def generate_test_data(df, test_date, time_gap, history_length_period, forward_peroid, generate_way):

    last_test_day_date = [(datetime.strptime(t,'%Y-%m-%d') - DT.timedelta(days=time_gap)).strftime('%Y-%m-%d') for t in test_date]

    if generate_way == 0: # use last day as feature, thus all future days have same features but different models
        # change it later.
        X_test = []
        t = prediction_day_index[c] - 1   

        train = df.iloc[t-peroid: t] # train data not included test data

        other_feature_cols = ['Weekly_Sales', 'IsHoliday', 'Temperature',
           'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4',
           'MarkDown5', 'CPI', 'Unemployment', 'Size', 'promotion',
           'Month', 'Year', 'day', 'dayofweek', 'y']
        
        X_test = [np.min(train.y), np.max(train.y), np.mean(train.y), np.std(train.y), stats.skew(train.y), stats.kurtosis(train.y)]
        Xtrain_other_features, _ = generate_sale_features(train[other_feature_cols])
        X_test += Xtrain_other_features

        
    else:
        
        X_test = [[] for _ in range(len(test_date))]
        Y_test = [[] for _ in range(len(test_date))]
        naive_test = [[] for _ in range(len(test_date))]
        ma_test = [[] for _ in range(len(test_date))]

        for i in range(len(test_date)): # Generate train data for each day model
            for d in df.Dept.unique():
                test_Res = df.loc[(df.Dept == d) & (df.Date == test_date[i])].Weekly_Sales
                if not len(test_Res):
                    continue
                df_d = df.loc[(df.Dept == d) & (df.Date <= last_test_day_date[i])]
                test_Res = test_Res.values[0]
                Y_test[i] += [test_Res] 
                t = -1
                train = df_d.iloc[t-history_length_period: t] # train data not included test data
                other_feature_cols = ['Weekly_Sales', 'IsHoliday', 'Temperature',
               'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4',
               'MarkDown5', 'CPI', 'Unemployment', 'Size', 'promotion',
               'Month', 'Year', 'day', 'dayofweek', 'y']

                Xtest_y_features = [np.min(train.y), np.max(train.y), np.mean(train.y), np.std(train.y), stats.skew(train.y), stats.kurtosis(train.y)]
                y_names = ['min_y', 'max_y', 'mean_y', 'std_y', 'skew_y', 'kurtosis_y']
                Xtest_other_features, other_names = generate_sale_features(train[other_feature_cols], forward_peroid) 
                X_test[i] += [Xtest_y_features + Xtest_other_features]
                test_feature_names = y_names + other_names 
                
                # save random walk res and moving average res
                naive_res = df.loc[(df.Dept == d) & (df.Date == last_test_day_date[i])].y
                if len(naive_res):
                    naive_test[i] += [naive_res.values[0]]
                else:
                    naive_test[i] += [-99999] #nan
                # monthly average 4 week
                previous_time = (datetime.strptime(last_test_day_date[i],'%Y-%m-%d') - DT.timedelta(days= 4*7)).strftime('%Y-%m-%d')
                ma_res = df.loc[(df.Dept == d) & (df.Date >= previous_time) & (df.Date <= last_test_day_date[i])].y
                if len(ma_res):
                    ma_test[i] += [np.mean(ma_res.values)]
                else:
                    ma_test[i] += [-99999] #nan
                    
    return X_test, Y_test, feature_names, naive_test, ma_test
            

In [150]:
# notice slide peroid in generating test data should be as long as possible! meaning the longest peroid data
# data1.Date.groupby(data1.Dept).count().unique(), longest is 143 peroid(Week)
# no sliding needed in here
X_test1, Y_test1, test_feature_names, naive_test, ma_test = generate_test_data(df = data1, 
                                      test_date = data1.Date.unique()[-4:], 
                                      time_gap = 7, 
                                      history_length_period = 143, 
                                      forward_peroid = [1,2,3], 
                                      generate_way = 1)

In [75]:
def rmse(y, yhat):
    return np.sqrt(((y - yhat)**2).mean())

def mape(pred, actual):
#     return np.mean(abs((actual - pred) / actual))
    return np.sum(abs(actual-pred)) / np.sum(actual)


In [ ]:
#     n = training_series.shape[0]
d = np.abs(np.diff( training_series) ).sum()/(n-1)

errors = np.abs(testing_series - prediction_series )
return errors.mean()/d

In [250]:
def mase(pred, actual, naive, ma): 
    # compare mean aboslute error of |actual - pred| with naive random walk or mean average
    index = []
    index += [i for i, x in enumerate(naive) if x == -99999]
    index += [i for i, x in enumerate(ma) if x == -99999]

    if index:
        actual = np.delete(actual, index)
        ma = np.delete(ma, index)
        naive = np.delete(naive, index)
        pred = np.delete(pred, index)

    residual = abs(actual - pred)
    ma_residual = abs(actual - ma)
    naive_residual = abs(actual - naive)
    return np.mean(residual) / np.mean(ma_residual), np.mean(residual) / np.mean(naive_residual)
           

In [186]:
import xgboost as xgb
import pickle

In [201]:
param = {'max_depth': 100, 'eta': 1, 'silent': 1,"objective":"reg:linear"}
param['nthread'] = 4
param['eval_metric'] = 'rmse'
num_round = 100
prediction_res = []
for i in range(len(X_train)):
    # 4 models for predicting 4 days
    dtrain = xgb.DMatrix(np.array(X_train[i]), label=np.array(Y_train[i]), feature_names=feature_names)
    bst = xgb.train(param, dtrain, num_round)
    dvalidation = xgb.DMatrix(np.array(X_validation[i]), feature_names=feature_names)
    validation_pred = bst.predict(dvalidation)
    print('cross validation mape ..' + str(mape(validation_pred, Y_validation[i])))
    print('cross validation rmse ..' + str(rmse(validation_pred, Y_validation[i])))
    dtest = xgb.DMatrix(np.array(X_test1[i]), feature_names=feature_names)
    ypred = bst.predict(dtest)
    prediction_res += [list(ypred)]
    print('final test mape ..' + str(mape(ypred, Y_test1[i])))
    print('final test mase ..' + str(
    mase(ypred, 
         np.array(Y_test1[i]), 
         np.array(naive_test[i]),
         np.array(ma_test[i])
        )))
    pickle.dump(bst, open("data1_day" + str(i) +".pickle.dat", "wb"))

cross validation mape ..0.022025163922687316
cross validation rmse ..5738.889095380739
final test mape ..0.10054226442482783
final test mase ..(1.3556084509206088, 1.5397721534859745)
cross validation mape ..0.02066579247687073
cross validation rmse ..5698.762228955335
final test mape ..0.0698581876861057
final test mase ..(1.4790737952611503, 1.5341176281602547)
cross validation mape ..0.02157458051066243
cross validation rmse ..5689.463526692383
final test mape ..0.05904189500231739
final test mase ..(2.901062498849261, 5.025765872378323)
cross validation mape ..0.020958633028560505
cross validation rmse ..5684.412508576465
final test mape ..0.07045011332124479
final test mase ..(1.1345030783125778, 1.6419634533877185)


In [258]:
# generate train and validation data for data1 and data2, because they will be trained seperately
# predict ['2012-10-05', '2012-10-12', '2012-10-19', '2012-10-26'] sales, 
# make sure that all id in test data have historical data in train data
X_train1, X_validation1, Y_train1, Y_validation1, feature_names = generate_backtest_data(df = data1.loc[data1.Date <= data1.Date.unique()[-5]], # training data 9864 records 
                                                                                     test_date = data1.Date.unique()[-4:], 
                                                                                     time_gap = 4 * 7, # week as unit, 4 weeks: 28 days 
                                                                                     slide_peroid = 20, # each has 8 week range 
                                                                                     slide_step = 4, 
                                                                                     forward_peroid = [1,2,3,4,5,6,7]) # slide back 1,2,3 weeks

['2012-09-07', '2012-09-14', '2012-09-21', '2012-09-28']


In [259]:
X_test1, Y_test1, test_feature_names, naive_test, ma_test = generate_test_data(df = data1, 
                                      test_date = data1.Date.unique()[-4:], 
                                      time_gap = 7, 
                                      history_length_period = 143, 
                                      forward_peroid = [1,2,3,4,5,6,7], 
                                      generate_way = 1)

In [263]:
np.array(X_train[i]).shape

(4530, 60)

In [264]:
param = {'max_depth': 100, 'eta': 1, 'silent': 1,"objective":"reg:linear"}
param['nthread'] = 4
param['eval_metric'] = 'rmse'
num_round = 100
prediction_res = []
for i in range(len(X_train)):
    # 4 models for predicting 4 days
    dtrain = xgb.DMatrix(np.array(X_train1[i]), label=np.array(Y_train1[i]), feature_names=feature_names)
    bst = xgb.train(param, dtrain, num_round)
    dvalidation = xgb.DMatrix(np.array(X_validation1[i]), feature_names=feature_names)
    validation_pred = bst.predict(dvalidation)
    print('cross validation mape ..' + str(mape(validation_pred, Y_validation1[i])))
    print('cross validation rmse ..' + str(rmse(validation_pred, Y_validation1[i])))
    dtest = xgb.DMatrix(np.array(X_test1[i]), feature_names=feature_names)
    ypred = bst.predict(dtest)
    prediction_res += [list(ypred)]
    print('final test mape ..' + str(mape(ypred, Y_test1[i])))
    print('final test mase ..' + str(
    mase(ypred, 
         np.array(Y_test1[i]), 
         np.array(naive_test[i]),
         np.array(ma_test[i])
        )))

cross validation mape ..0.0451212611774553
cross validation rmse ..8485.227061072057
final test mape ..0.10058386706830091
final test mase ..(0.950018458492044, 1.1365753360549893)
cross validation mape ..0.04142084108196895
cross validation rmse ..8324.140609583856
final test mape ..0.06791809101479906
final test mase ..(0.9505787455880899, 1.186108122841672)
cross validation mape ..0.042291577081870806
cross validation rmse ..8346.298170579334
final test mape ..0.06821256853018214
final test mase ..(1.0476640908800874, 1.4290436268264048)
cross validation mape ..0.043170552792823946
cross validation rmse ..8428.739660507961
final test mape ..0.07207139225720414
final test mase ..(0.8979836069818379, 1.3660908374764993)


In [ ]:
# LSTM pre-processing
# difference the timeseries to be staionary?
# normalization or scale Min_Max_Scale
# validation set in LSTM set-up

In [317]:
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return diff

In [266]:
data = pd.read_csv('./new_data.csv')

In [330]:
data = pd.concat([data1, data2])

In [268]:
data.describe()

,Store,Dept,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Size,promotion,Month,Year,day,dayofweek,y
count,16440.000000,16440.000000,16440.000000,16440.000000,16440.000000,16440.000000,16440.000000,16440.000000,16440.000000,16440.000000,16440.000000,16440.000000,16440.000000,16440.000000,16440.0,16440.000000,16440.000000,16440.000000,16440.0,16440.000000
mean,24.973844,45.078650,20592.981457,0.070255,63.625909,3.485616,-61686.699336,-73818.639816,-72194.903120,-74695.921838,-62089.654824,178.330466,7.800676,140975.145985,0.0,6.463686,2010.978224,15.678345,4.0,14160.816354
std,6.318487,31.745177,30850.137119,0.255585,19.149220,0.449833,51667.207726,45874.391596,45989.171665,45096.329064,50274.315070,39.186836,0.840250,79735.416821,0.0,3.237655,0.796389,8.751370,0.0,14902.986229
min,20.000000,1.000000,-798.000000,0.000000,20.390000,2.699000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,126.064000,6.895000,39690.000000,0.0,1.000000,2010.000000,1.000000,4.0,103.527212
25%,20.000000,17.000000,1537.680000,0.000000,50.520000,3.049000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,129.845967,7.274000,39690.000000,0.0,4.000000,2010.000000,8.000000,4.0,3688.954121
50%,20.000000,40.000000,8059.405000,0.000000,64.020000,3.583000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,204.648780,7.484000,203742.000000,0.0,6.000000,2011.000000,16.000000,4.0,7553.595042
75%,33.000000,79.000000,26817.895000,0.000000,76.030000,3.842000,470.510000,-0.990000,2.360000,-99999.000000,1641.910000,209.498613,8.187000,203742.000000,0.0,9.000000,2012.000000,23.000000,4.0,20884.170913
max,33.000000,99.000000,422306.250000,1.000000,100.140000,4.468000,58928.520000,97740.990000,101378.790000,53603.990000,35675.620000,216.151590,10.115000,203742.000000,0.0,12.000000,2012.000000,31.000000,4.0,59081.781274


In [335]:
# normalization data
data.replace(-99999, 0, inplace=True)


In [336]:
# pad 
# check all dept Date range/length , for date length < 33, we will remove it.
lengths = data.Date.groupby([data.Store, data.Dept]).count().reset_index()
length_range = lengths['Date'].value_counts().reset_index()
np.percentile(days, 25)

35.75

In [337]:
remove_ids =[str(i[0]) + '_' + str(i[1]) for i in lengths.loc[lengths.Date<35][['Store', 'Dept']].values]

In [297]:
remove_ids

['20_47', '20_96', '33_23', '33_31', '33_32', '33_44', '33_49']

In [338]:
data = data.loc[~data.id.isin(remove_ids)]

In [339]:
# univariate data
test_date

array(['2012-10-05', '2012-10-12', '2012-10-19', '2012-10-26'],
      dtype=object)

In [382]:
train, test = data.loc[data.Date < test_date[0]], data.loc[data.Date >= test_date[0]]

In [341]:
test.id.unique() == train.id.unique()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [347]:
def standardlization(X):
    mean = np.mean(X, axis = 0)
    X -=  mean
    X /= np.std(X, axis = 0)
    return np.array(X)

In [345]:
from sklearn.preprocessing import scale

In [350]:
from sklearn.preprocessing import StandardScaler

In [358]:
scaler = StandardScaler()
raw= np.array(raw).reshape(-1, 1)
scaler.fit(raw)
m = scaler.transform(raw)

In [307]:
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [308]:
# generate univariate data timseries for keras
# all pad as 143 long, fill zero if current value list shorter than 143
# generate train value with slide peroid and slide steps, generate test data with all historical value dataset
# more focused on the LSTM Structure design, compare LSTM with CNN

In [383]:
# X_train, Y_train, X_validation, Y_validation
pd.options.mode.chained_assignment = None
slide_period = 20
slide_step = 4
df = train
T, D = 143 , 1
X_train, Y_train = [], []
X_validation, Y_validation = [], []

test_date = data.Date.unique()[-4:]
time_gap = 4 * 7
last_test_day_date = [(datetime.strptime(t,'%Y-%m-%d') - DT.timedelta(days=time_gap)).strftime('%Y-%m-%d') for t in test_date]

for i in range(len(test_date)):
    for d in df.Dept.unique():
        df_d = df.loc[(df.Dept == d) & (df.Date <= last_test_day_date[i])]
        t = -1 
        
        df_d['scaled_y'] = scale(df_d.y.values)
        while -(t - slide_peroid) <= len(df_d):
            
            validation_data = df_d.iloc[t-slide_peroid-1 : t+1].scaled_y.values
            train_data = df_d.iloc[t-slide_peroid: t].scaled_y.values
            
            # pad sequence here...
            
            X_validation += [list(validation_data)]
            X_train += [list(train_data)]
            
            Y_train += [df_d.iloc[t-1].scaled_y]

            Y_validation += [df_d.iloc[t].scaled_y]
            t -= slide_step

In [390]:
df_d.iloc[t-1].scaled_y

-1.2347911434735876

In [376]:
df_d

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,...,Type,Size,promotion,Month,Year,day,dayofweek,id,y,scaled_y
1430,20,11,2010-02-05,33693.80,0,25.92,2.784,0.00,0.00,0.00,...,A,203742,0,2,2010,5,4,20_11,34560.213601,1.578950
1431,20,11,2010-02-12,33919.29,1,22.12,2.773,0.00,0.00,0.00,...,A,203742,0,2,2010,12,4,20_11,31393.670194,1.348139
1432,20,11,2010-02-19,26807.27,0,25.43,2.745,0.00,0.00,0.00,...,A,203742,0,2,2010,19,4,20_11,28318.352102,1.123978
1433,20,11,2010-02-26,25207.18,0,32.32,2.754,0.00,0.00,0.00,...,A,203742,0,2,2010,26,4,20_11,25116.883012,0.890621
1434,20,11,2010-03-05,21309.54,0,31.75,2.777,0.00,0.00,0.00,...,A,203742,0,3,2010,5,4,20_11,23045.844352,0.739663
1435,20,11,2010-03-12,21649.51,0,43.82,2.818,0.00,0.00,0.00,...,A,203742,0,3,2010,12,4,20_11,21785.626633,0.647805
1436,20,11,2010-03-19,21913.11,0,47.32,2.844,0.00,0.00,0.00,...,A,203742,0,3,2010,19,4,20_11,21291.831953,0.611812
1437,20,11,2010-03-26,19174.25,0,50.49,2.854,0.00,0.00,0.00,...,A,203742,0,3,2010,26,4,20_11,21618.199738,0.635601
1438,20,11,2010-04-02,23569.73,0,51.00,2.850,0.00,0.00,0.00,...,A,203742,0,4,2010,2,4,20_11,21047.273584,0.593986
1439,20,11,2010-04-09,29741.36,0,65.10,2.869,0.00,0.00,0.00,...,A,203742,0,4,2010,9,4,20_11,20350.264949,0.543181
